In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import PreventUpdate to prevent callback from firing
from dash.exceptions import PreventUpdate

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import AnimalShelter Module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
shelter = AnimalShelter() # create instance of AnimalShelter class containing credentials

# Fetch data from database
def fetch_data(query={}):
    """Fetch data from MongoDB and return as a DataFrame."""
    df = pd.DataFrame.from_records(shelter.read(query))
    df.drop(columns=['_id'], inplace=True)
    return df

# Initial data load
df = fetch_data()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add the logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    dcc.Store(id='reset-trigger', data=0),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        children=[
            html.A( # Add the Logo and embedded link to the dashboard
                html.Img(src='data:image/svg+xml;base64,{}'.format(encoded_image.decode()), style={'height':'200px', 'width':'200px'}),
                href='https://www.snhu.edu',
                target='_blank'
            ),
            html.H1('Grazioso Salvare')
        ],
        style={'display': 'flex', 'align-items': 'center'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
        data=df.to_dict('records'),
        sort_action='native',
        sort_mode='multi',
        column_selectable='single', # Enable single column selection
        row_selectable='single', # Enable single row selection
        selected_columns=[], # Default selected column is None
        selected_rows=[0], # Default selected row is First row
        page_action='native', # Pagination
        page_current=0, # Set Active Page default
        page_size=10, # Limit active page size
        style_table={'overflowX': 'auto'}
    ),
    html.Div(
        children=[
            dcc.Dropdown(
                id='rescue-type-dropdown',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
                ],
                style={'width': '300px'}
            ),
            dcc.Dropdown(
                id='field-dropdown',
                options=[{'label': i, 'value': i} for i in df.columns],
                style={'width': '300px'}
            ),
            dcc.Dropdown(
                id='value-dropdown',
                style={'width': '300px'}
            ),
            html.Button('Reset Dashboard', id='reset-button', n_clicks=0),
        ],
        style={'display': 'flex', 'justify-content': 'space-between'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'padding-right': '10px'}
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.H2("Interactive Dashboard produced by: Joshua Lewis"),
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('reset-trigger', 'data'),
    [Input('reset-button', 'n_clicks')])
def reset_dashboard(n):
    return n

@app.callback(
    [Output('datatable-id', 'data'),
     Output('rescue-type-dropdown', 'value'),
     Output('field-dropdown', 'value'),
     Output('value-dropdown', 'value')],
    [Input('reset-trigger', 'data'),  # This triggers the reset
     Input('rescue-type-dropdown', 'value'),
     Input('field-dropdown', 'value'),
     Input('value-dropdown', 'value')])
def update_dashboard(n, rescue_type, selected_field, selected_value):
    """Update dashboard based on user interactions."""
    dff = df  # Start with the full data
    
    if n:
        rescue_type = None
        selected_field = None
        selected_value = None

    if rescue_type:  # Check if rescue_type is not None
        try:
            dff = dff[dff['rescue_type'] == rescue_type]
        except KeyError:
            print(f"KeyError: 'rescue_type' not found in DataFrame.")
            raise PreventUpdate
    if selected_field and selected_value:  # Check if both selected_field and selected_value are not None
        try:
            dff = dff[dff[selected_field] == selected_value]
        except KeyError:
            print(f"KeyError: '{selected_field}' or '{selected_value}' not found in DataFrame.")
            raise PreventUpdate

    data = dff.to_dict('records')
    return data, rescue_type, selected_field, selected_value

@app.callback(
    Output('value-dropdown', 'options'),
    [Input('field-dropdown', 'value')])
def update_value_dropdown(selected_field):
    """Update value dropdown based on selected field."""
    try:
        unique_values = df[selected_field].unique()
    except KeyError:
        print(f"KeyError: '{selected_field}' not found in DataFrame.")
        raise PreventUpdate
    options = [{'label': v, 'value': v} for v in unique_values]
    return options

@app.callback(
    Output('graph-id', "children"),
    [Input('reset-trigger', 'data'),  # This triggers the reset
     Input('datatable-id', "derived_virtual_data"),
     Input('field-dropdown', 'value'),
     Input('value-dropdown', 'value')])
def update_graphs(n, viewData, selected_field, selected_value):
    """Update pie chart based on selected field or reset the graph."""
    if n:  # Reset trigger has been activated
        return []  # Return an empty list to clear the graph
    elif viewData is None:
        raise PreventUpdate

    dff = pd.DataFrame.from_dict(viewData)
    if selected_field:  # Main filter is selected
        if selected_value:  # Subfilter is selected
            try:
                dff = dff[dff[selected_field] == selected_value]
            except KeyError:
                print(f"KeyError: '{selected_field}' or '{selected_value}' not found in DataFrame.")
                raise PreventUpdate
        fig = px.pie(dff, names=selected_field, title=f'Distribution of {selected_field}')
    else:  # Main filter is not selected
        fig = px.histogram(dff, title='Distribution of Data')

    return [
        dcc.Graph(figure=fig)
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    """Update map based on selected row on data table."""
    if viewData is None: # Check if data is being passed from DataFrame in callback
        print("Debugging: viewData is None, check if DataTable is populated.")
        raise PreventUpdate

    dff = pd.DataFrame.from_dict(viewData) # Convert viewData to DataFrame

    # If selected_rows is None or empty, default to the first row
    if not selected_rows:
        selected_rows = [0]

    try:
        row = selected_rows[0] # Default row
        center = [dff.iloc[row, 13], dff.iloc[row, 14]] # Coordinates
        tooltip = dff.iloc[row, 4] # Breed
        popup = dff.iloc[row, 9] # Name
    except IndexError as e:
        print(f"Debugging: IndexError - {e}")
        raise PreventUpdate
    except KeyError as e:
        print(f"Debugging: KeyError - {e}")
        raise PreventUpdate

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=center, zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=center, children=[
                    dl.Tooltip(tooltip),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(popup)
                    ])
                ])
            ]
        )
    ]

app.run_server(mode='external', debug=True)

ModuleNotFoundError: No module named 'dash_leaflet'